In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import cvxpy

In [2]:
xor_table = pd.DataFrame({
    'x1': [-1, -1, +1, +1],
    'x2': [-1, +1, -1, +1],
    'y': [-1, 1, 1, -1],
})
xs = xor_table[['x1', 'x2']].values
ds = xor_table[['y']].values
display(xs, ds)
xor_table

array([[-1, -1],
       [-1,  1],
       [ 1, -1],
       [ 1,  1]])

array([[-1],
       [ 1],
       [ 1],
       [-1]])

,x1,x2,y
0,-1,-1,-1
1,-1,1,1
2,1,-1,1
3,1,1,-1


### Ядро

$K(x, x_i) = \varphi^T(x) \varphi(x_i) = \Sigma_{j=0}^{m1} \varphi_j(x) \varphi_j(x_i)$

полиномиальное ядро
$K = (x^Tx_i + 1)^p$

In [5]:
K = (xs @ xs.T + 1) ** 2
C = 1
K

array([[9, 1, 1, 1],
       [1, 9, 1, 1],
       [1, 1, 9, 1],
       [1, 1, 1, 9]])

[example 1](https://github.com/arkm97/svm-from-scratch/blob/master/SVM_from_scratch.ipynb),
[example 2](https://github.com/ur6yr/support-vector-machine/blob/master/svm.ipynb), 
[3](https://github.com/kenextra/SVMAlgorithm/blob/master/SVM.py),
[convex quadratic programs](http://cvxopt.org/userguide/coneprog.html#quadratic-programming)

### Прямая задача (primal problem)
- минимизировать целевую функцию $\Phi(w) = \frac{1}{2}w^Tw$ относительно $w$
- при ограничении $d_i(w^Tx_i + b) \ge 1$ для $i= 1,2,...,N$

### Двойственная задача (dual problem)
- максимизировать $Q(\alpha) = \Sigma_{i=1}^N\alpha_i - \frac{1}{2}\Sigma_{i=1}^N\Sigma_{j=1}^N\alpha_i\alpha_j d_i d_j x_i^T x_j$ относительно $\alpha$
- при ограничениях (subject to)
  - $\Sigma_{i=1}^N\alpha_i d_i = 0$
  - $0 \le \alpha_i \le C$ для $i = 1, 2, ..., N$

In [86]:
a = cvxpy.Variable(shape=ds.shape)
P = (ds @ ds.T) * K
Q = cvxpy.sum(a) - 1/2 * cvxpy.quad_form(a, P)

objective = cvxpy.Maximize(Q)
constraints = [
    cvxpy.sum(cvxpy.multiply(a, ds)) == 0,
    0 <= a,
    a <= C,
]
problem = cvxpy.Problem(objective, constraints)
result = problem.solve()
a = a.value
Q = Q.value
w_norm = (2*Q)**0.5
print(a, Q, w_norm)

[[0.125]
 [0.125]
 [0.125]
 [0.125]] [[0.25]] [[0.70710678]]


уравнение гиперплоскости решений

$\Sigma_{j=1}^{m1} w_j \varphi_j(x) + b = 0$

$w^T \varphi(x) = 0$

In [96]:
K @ (a*ds * w_norm)

array([[-0.70710678],
       [ 0.70710678],
       [ 0.70710678],
       [-0.70710678]])

In [70]:
def predict(X):
    rs = X @ w + b
    return rs